In [1]:
pip install billboard.py

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lyricsgenius

     |████████████████████████████████| 92kB 689kB/s eta 0:00:01
  Found existing installation: beautifulsoup4 4.8.0
    Uninstalling beautifulsoup4-4.8.0:
      Successfully uninstalled beautifulsoup4-4.8.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import billboard as bb
import lyricsgenius

In [4]:
genius = lyricsgenius.Genius("JxIIf2vA02Oopvz2JRUbC6iysXRZnNEHN0j84a16uLsB8niafpsNZPR1SvcSxu8r")
genius.verbose = False # Turn off status messages
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = False # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Remix)", "(Live)"] # Exclude songs with these words in their title

In [5]:
def create_songs_df(chart_name):
    top100 = bb.ChartData(chart_name)
    songs = top100.entries
    dataframe = {"Rank": [], "Title": [], "Artist": []}
    for song in songs:
        dataframe["Title"].append(song.title)
        dataframe["Artist"].append(song.artist)
        dataframe["Rank"].append(song.rank)
    df = pd.DataFrame(dataframe)
    return df

In [6]:
top100 = create_songs_df("hot-100")
country = create_songs_df("country-songs")
rock = create_songs_df("rock-songs")
pop = create_songs_df("pop-songs")
print(top100)

    Rank              Title                                            Artist
0      1            Circles                                       Post Malone
1      2  Someone You Loved                                     Lewis Capaldi
2      3       Good As Hell                                             Lizzo
3      4           Memories                                          Maroon 5
4      5            Roxanne                                    Arizona Zervas
..   ...                ...                                               ...
95    96    Hot Girl Summer  Megan Thee Stallion, Nicki Minaj & Ty Dolla $ign
96    97       Lonely Child                        YoungBoy Never Broke Again
97    98   Into The Unknown                               Panic! At The Disco
98    99      Show Yourself                   Idina Menzel & Evan Rachel Wood
99   100           Homesick                                        Kane Brown

[100 rows x 3 columns]


In [7]:
def try_load_lyrics(title,artist):
    try:
        return genius.search_song(title, artist).lyrics
    except:
        return "Failed to Find Lyrics"

In [8]:
def populate_chart_with_lyrics(chart):
    chart["lyrics"] = chart.apply(lambda x: try_load_lyrics(x["Title"],x["Artist"]), axis = 1)
    return chart

In [9]:
top100 = populate_chart_with_lyrics(top100)
display(top100)
len(top100[top100['lyrics'] == 'Failed to Find Lyrics'])

,Rank,Title,Artist,lyrics
0,1,Circles,Post Malone,"Oh, oh, oh\nOh, oh, oh\nOh, oh, oh, oh, oh\n..."
1,2,Someone You Loved,Lewis Capaldi,"I'm going under, and this time, I fear there's..."
2,3,Good As Hell,Lizzo,"I do my hair toss, check my nails\nBaby, how y..."
3,4,Memories,Maroon 5,Here's to the ones that we got\nCheers to the ...
4,5,Roxanne,Arizona Zervas,All for the 'Gram\nBitches love the 'Gram\nOh ...
5,6,Lose You To Love Me,Selena Gomez,You promised the world and I fell for it\nI pu...
6,7,Senorita,Shawn Mendes & Camila Cabello,I love it when you call me señorita\nI wish I ...
7,8,"10,000 Hours",Dan + Shay & Justin Bieber,"Do you love the rain, does it make you dance\n..."
8,9,Truth Hurts,Lizzo,Why're men great 'til they gotta be great?\nWo...
9,10,No Guidance,Chris Brown Featuring Drake,My name is Devvon Terrell\nThis is a raw sessi...


1

In [11]:
pip install spotipy

  Created wheel for spotipy: filename=spotipy-2.4.4-cp37-none-any.whl size=12335 sha256=5555e0c0b366b5d319b204997a81b8c6305251e525c4df45b13cff6025b59c6c
  Stored in directory: /home/jovyan/.cache/pip/wheels/76/28/19/a86ca9bb0e32dbd4a4f580870250f5aeef852870578e0427e6
Successfully built spotipy
Note: you may need to restart the kernel to use updated packages.


In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id = "ee9a1f8e2f634959967d52ed9fad54de", client_secret = "934a6bbc90e74664b502638f6f5fc92b")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [43]:
answer = sp.search("Ransom", type="track", limit=1)
display(answer["tracks"]["items"][0])
track = sp.track(answer["tracks"]["items"][0]["id"])
track["popularity"]

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Ga1P7PMIsmqEZqhYZQgDo'},
    'href': 'https://api.spotify.com/v1/artists/4Ga1P7PMIsmqEZqhYZQgDo',
    'id': '4Ga1P7PMIsmqEZqhYZQgDo',
    'name': 'Lil Tecca',
    'type': 'artist',
    'uri': 'spotify:artist:4Ga1P7PMIsmqEZqhYZQgDo'}],
  'available_markets': ['AD',
   'AE',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PS',
   'PT',
   'PY',
   'QA',
   'RO',
   'SA',
   'SE',
   'SG',
   'SK',
  

92

In [44]:
def get_spotify_popularity(title):
    title = title.replace("$","s")
    answer = sp.search(title, type="track", limit = 1)
    track = sp.track(answer["tracks"]["items"][0]["id"])
    return track["popularity"]

In [45]:
def populate_chart_with_spotify_popularity(chart):
    chart["spotify_popularity"] = chart.apply(lambda x: get_spotify_popularity(x["Title"]), axis = 1)
    return chart

In [46]:
top100 = populate_chart_with_spotify_popularity(top100)
display(top100)

,Rank,Title,Artist,lyrics,spotify_popularity
0,1,Circles,Post Malone,"Oh, oh, oh\nOh, oh, oh\nOh, oh, oh, oh, oh\n...",99
1,2,Someone You Loved,Lewis Capaldi,"I'm going under, and this time, I fear there's...",96
2,3,Good As Hell,Lizzo,"I do my hair toss, check my nails\nBaby, how y...",86
3,4,Memories,Maroon 5,Here's to the ones that we got\nCheers to the ...,100
4,5,Roxanne,Arizona Zervas,All for the 'Gram\nBitches love the 'Gram\nOh ...,96
5,6,Lose You To Love Me,Selena Gomez,You promised the world and I fell for it\nI pu...,98
6,7,Senorita,Shawn Mendes & Camila Cabello,I love it when you call me señorita\nI wish I ...,96
7,8,"10,000 Hours",Dan + Shay & Justin Bieber,"Do you love the rain, does it make you dance\n...",95
8,9,Truth Hurts,Lizzo,Why're men great 'til they gotta be great?\nWo...,91
9,10,No Guidance,Chris Brown Featuring Drake,My name is Devvon Terrell\nThis is a raw sessi...,87
